In [1]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import FAISS
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from pathlib import Path
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langchain_huggingface import HuggingFaceEmbeddings
import os
from tqdm.autonotebook import tqdm

C:\Users\crowl\AppData\Local\Temp\ipykernel_1768\1591546265.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
llm = ChatOllama(model="llama3.2")

#=============================================================================
# 1) Identify database path 
#=============================================================================
matt_path=Path(os.getcwd())
root_path=matt_path.parents[0]
data_path=root_path.joinpath('data')
db_path=root_path.joinpath(r'data\vectorstore\db_faiss')
word_path=root_path.joinpath(r'data\apra_standards\word')

#=============================================================================
# 3) Load vectorstore, retriever and LLM
#=============================================================================
embedding_model='sentence-transformers/all-MiniLM-L6-v2'
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
vectorstore = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": "What are the key elements in the definition of Effective Maturity?"},
    config=config,
)
print(result["answer"])

The context does not provide a direct answer to what are the key elements in the definition of Effective Maturity. However, it can be inferred that Effective Maturity is related to the maturity or settlement date of financial instruments and exposures. 

If you would like, I can tell you about other parts of the text regarding maturity but without giving you an explicit definition.


In [7]:
result = app.invoke(
    {"input": "Yes, please tell me"},
    config=config,
)
print(result["answer"])

The context mentions several aspects related to maturity, including:

1. Residual maturity: Refers to the remaining time until a claim or exposure is settled.
2. Time to final payment: Refers to the time period from the current date to the expected settlement date of an asset or exposure.
3. Contractual maturity: Refers to the contractual date specified in the terms and conditions of a financial instrument or exposure.
4. Longest maturity of assets: Refers to the longest possible remaining time that an ADI (Authorized Deposit Institution) would be exposed to potential losses from securitized exposures.

These aspects are mentioned as factors to consider when determining the maturity of a securitization exposure, but without providing a single definition or key elements.


In [8]:
result = app.invoke(
    {"input": "I want you to act as an expert in Australian banking regulations preparing a Q&A list for a website. Can you please give me a list of 34 questions someone working as either an accountant, lawyer or actuary may ask regarding Australian Prudential banking regulations. I want each question to refer to a randomly selected standard chosen without replacement."},
    config=config,
)
print(result["answer"])

Here are 34 questions that someone working in accounting, law, or actuarial roles may ask regarding Australian prudential banking regulations, with each question referring to a randomly selected standard chosen without replacement:

1. What is the main purpose of the Basel III regulatory framework, as implemented in Australia?
(Referenced Standard: Basel III)
2. Can you explain the difference between an Australian Deposit-Taking Institution (DTI) and a Non-Deposit Taking Institution (NDTI)?
(Referenced Standard: APS 210 Liquidity)
3. How do Australian banks report their exposure to credit risk using the Internal Ratings-Based Approach (IRB)?
(Referenced Standard: APS 112 Credit Risk Management)
4. What is the definition of a Systemically Important Financial Institution (SIFI) under Australian prudential regulations?
(Referenced Standard: APS 209 Capital Adequacy)
5. How do Australian banks calculate their Basel III capital requirements for Tier 1 and Tier 2 capital?
(Referenced Standar

In [9]:
result = app.invoke(
    {"input": "Can you please reform this output into a form that is easily converted into a CSV file?"},
    config=config,
)
print(result["answer"])

Here's the reformatted text in a format that can be easily converted to a CSV file:

```
Key principles
An ADI must make accurate, high quality and timely public disclosures of information on its: 
risk profile;
risk management;
capital adequacy;
capital instruments;
remuneration practices; 
and,
where applicable, on its:
leverage ratio;
liquidity coverage ratio (LCR) and net stable funding ratio (NSFR); 
global systemically important bank (G-SIB) indicators,

An ADI’s public disclosures must be consistent with the scope and complexity of its operations and the sophistication of its risk management systems and processes.

Definitions
‘accounting standards’ means Australian Accounting Standards;
‘annual directors’ report’ is the directors’ report for a financial year required under the Corporations Act;
‘annual financial report’ has the meaning given in the Corporations Act;
‘AT1’ refers to Additional Tier 1 Capital;
‘the Basel Committee’ refers to the Basel Committee on Banking Supervi

### Test and Evaluation
Testing manually curated questions and answers.

In [3]:
import chardet
import pandas as pd
qa_filepath=data_path.joinpath("question-answers.csv")

# Read the file in binary mode to detect encoding
with open(qa_filepath, "rb") as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")
    
# load questions
df = pd.read_csv(qa_filepath, encoding=encoding)

df.head()

Detected encoding: MacRoman


,#,Question,Answer
0,1,What is the main purpose of the Basel III regu...,The Basel III capital reforms are internationa...
1,2,Can you explain the difference between an Aust...,An Authorised Deposit-taking Institutions (ADI...
2,3,How do Australian banks report their exposure ...,IRB banks must compare regulatory expected los...
3,4,What is the definition of a Systemically Impor...,The too-big-to-fail problem refers to the fact...
4,5,How do Australian banks calculate their Basel ...,The capital adequacy ratio is calculated by ad...


In [7]:
config = {"configurable": {"thread_id": "abc123"}}

# Use the QA Agent to get answers for the list of questions
questions = df["Question"]
answers = []

for q in questions:
    answer = app.invoke({"input": q},config=config)["answer"] if result else ""
    answers.append(answer)

print("Number of answers:", len(answers))

Number of answers: 80


In [8]:
# save the answers to csv
df["RAG 2"] = answers
df.to_csv(data_path.joinpath("answers_rag2.csv"))

In [9]:
# Examine the answers to the questions
i=0
for question, answer in zip(questions, answers):
    i += 1
    print(i)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print()

1
Question: What is the main purpose of the Basel III regulatory framework, as implemented in Australia?
Answer: The main purpose of the Basel III regulatory framework, as implemented in Australia, is to strengthen the capital and liquidity requirements for banks and other financial institutions. The framework aims to:

1. **Enhance bank stability**: By increasing the minimum levels of common equity tier 1 (CET1) capital and total loss-absorbing capacity (TLAC), Basel III seeks to improve the resilience of banks against potential losses.
2. **Reduce risk-taking**: By requiring banks to hold more capital and liquidity, Basel III aims to reduce the risk that banks may take on excessive debt or invest in high-risk assets.
3. **Improve financial stability**: The framework also aims to prevent bank failures by ensuring that banks have sufficient buffers to absorb potential losses.

Key changes introduced by Basel III, as implemented in Australia, include:

1. A minimum CET1 ratio of 7% (com